<a href="https://colab.research.google.com/github/apriandito/nowcasting/blob/main/3_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge, Lasso, BayesianRidge, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go

Get API Key: http://alphavantage.co/

In [3]:
# Ambil data dari API
API_KEY = '8V7F020H9WXQRHLQ'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol=IDR&apikey={API_KEY}&outputsize=full"
response = requests.get(url)
data = response.json()
timeseries = data.get("Time Series FX (Daily)", {})
timeseries

{'2025-05-01': {'1. open': '16674.00000',
  '2. high': '16674.00000',
  '3. low': '16594.00000',
  '4. close': '16597.00000'},
 '2025-04-30': {'1. open': '16782.00000',
  '2. high': '16782.00000',
  '3. low': '16589.00000',
  '4. close': '16594.00000'},
 '2025-04-29': {'1. open': '16857.00000',
  '2. high': '16857.00000',
  '3. low': '16753.00000',
  '4. close': '16754.00000'},
 '2025-04-28': {'1. open': '16824.00000',
  '2. high': '16863.00000',
  '3. low': '16819.00000',
  '4. close': '16849.00000'},
 '2025-04-25': {'1. open': '16799.00000',
  '2. high': '16879.00000',
  '3. low': '16799.00000',
  '4. close': '16824.00000'},
 '2025-04-24': {'1. open': '16875.00000',
  '2. high': '16884.00000',
  '3. low': '16849.00000',
  '4. close': '16864.00000'},
 '2025-04-23': {'1. open': '16857.00000',
  '2. high': '16884.00000',
  '3. low': '16844.00000',
  '4. close': '16859.00000'},
 '2025-04-22': {'1. open': '16803.00000',
  '2. high': '16864.00000',
  '3. low': '16799.00000',
  '4. close': 

In [4]:
# Konversi ke DataFrame
df = pd.DataFrame.from_dict(timeseries, orient='index')
df.columns = ['Open', 'High', 'Low', 'Close']
df.index = pd.to_datetime(df.index)
df = df.sort_index()
df = df.astype(float)
df

,Open,High,Low,Close
2014-11-07,12120.0,12170.0,12115.0,12135.0
2014-11-10,12170.0,12170.0,12135.0,12150.0
2014-11-11,12200.0,12200.0,12200.0,12200.0
2014-11-12,12200.0,12218.0,12142.0,12197.0
2014-11-13,12197.0,12210.0,12142.0,12200.0
...,...,...,...,...
2025-04-25,16799.0,16879.0,16799.0,16824.0
2025-04-28,16824.0,16863.0,16819.0,16849.0
2025-04-29,16857.0,16857.0,16753.0,16754.0
2025-04-30,16782.0,16782.0,16589.0,16594.0


In [18]:
# Buat salinan dan fitur tambahan
df_feat = df.copy()

# Fitur hari ini (Open hari ini) — masih diperbolehkan
df_feat['Open_today'] = df_feat['Open']

# Fitur hari sebelumnya (lag 1)
df_feat['Close_t-1'] = df_feat['Close'].shift(1)
df_feat['Open_t-1'] = df_feat['Open'].shift(1)
df_feat['High_t-1'] = df_feat['High'].shift(1)
df_feat['Low_t-1'] = df_feat['Low'].shift(1)

# Moving Averages dari harga Close historis (hindari leakage)
df_feat['MA_3'] = df_feat['Close'].shift(1).rolling(window=3).mean()
df_feat['MA_5'] = df_feat['Close'].shift(1).rolling(window=5).mean()

# Rolling Standard Deviation (historis)
df_feat['STD_3'] = df_feat['Close'].shift(1).rolling(window=3).std()
df_feat['STD_5'] = df_feat['Close'].shift(1).rolling(window=5).std()

# Rate of Change (ROC) historis
df_feat['ROC_1'] = df_feat['Close'].shift(1).pct_change(periods=1)
df_feat['ROC_3'] = df_feat['Close'].shift(1).pct_change(periods=3)

# Momentum historis
df_feat['Momentum_1'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(2)
df_feat['Momentum_3'] = df_feat['Close'].shift(1) - df_feat['Close'].shift(4)

# Hapus baris dengan missing values
df_feat.dropna(inplace=True)


In [19]:
# Pisahkan baris terakhir untuk prediksi real-time
latest_row = df_feat.iloc[[-1]]
df_feat = df_feat.iloc[:-1]

# Siapkan fitur dan target
# Siapkan fitur dan target (tambahkan fitur baru)
X = df_feat[[
    'Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1',
    'MA_3', 'MA_5', 'STD_3', 'STD_5', 'ROC_1', 'ROC_3', 'Momentum_1', 'Momentum_3'
]]

y = df_feat['Close']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

In [20]:
# Lihat Train Data
X_train

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1,MA_3,MA_5,STD_3,STD_5,ROC_1,ROC_3,Momentum_1,Momentum_3
2014-11-14,12200.0,12200.0,12197.0,12210.0,12142.0,12199.000000,12176.4,1.732051,31.421330,0.000246,0.004115,3.0,50.0
2014-11-17,12210.0,12210.0,12200.0,12218.0,12200.0,12202.333333,12191.4,6.806859,23.660093,0.000820,0.000820,10.0,10.0
2014-11-18,12202.0,12202.0,12210.0,12210.0,12185.0,12204.000000,12201.8,5.291503,4.919350,-0.000655,0.000410,-8.0,5.0
2014-11-19,12145.0,12145.0,12202.0,12210.0,12065.0,12185.666667,12190.8,35.444793,26.051871,-0.004671,-0.004508,-57.0,-55.0
2014-11-20,12145.0,12145.0,12145.0,12145.0,12120.0,12164.000000,12180.4,32.908965,32.531523,0.000000,-0.005324,0.0,-65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-21,15360.0,15354.0,15360.0,15375.0,15339.0,15376.333333,15372.8,33.620430,24.833445,-0.000391,-0.000391,-6.0,-6.0
2023-03-22,15345.0,15339.0,15360.0,15360.0,15329.0,15351.000000,15365.6,10.816654,28.918852,-0.000977,-0.004930,-15.0,-76.0
2023-03-23,15344.0,15339.0,15345.0,15345.0,15339.0,15344.000000,15361.4,8.660254,31.357615,0.000000,-0.001367,0.0,-21.0
2023-03-24,15345.0,15339.0,15344.0,15345.0,15339.0,15339.000000,15346.2,0.000000,10.084642,0.000000,-0.000977,0.0,-15.0


In [21]:
# Lihat Test Data
X_test

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1,MA_3,MA_5,STD_3,STD_5,ROC_1,ROC_3,Momentum_1,Momentum_3
2023-03-28,15170.0,15154.0,15185.0,15185.0,15149.0,15214.000000,15264.0,108.282039,102.713193,0.000330,-0.012061,5.0,-185.0
2023-03-29,15085.0,15084.0,15170.0,15189.0,15059.0,15129.000000,15213.0,39.051248,118.289898,-0.004619,-0.016624,-70.0,-255.0
2023-03-30,15059.0,15059.0,15085.0,15109.0,15054.0,15099.000000,15157.0,49.244289,109.692753,-0.001657,-0.005941,-25.0,-90.0
2023-03-31,15060.0,15044.0,15059.0,15090.0,15039.0,15062.333333,15098.0,20.207259,50.916598,-0.000996,-0.007259,-15.0,-110.0
2023-04-03,15060.0,14989.0,15060.0,15060.0,14944.0,15030.666667,15066.0,36.855574,60.270225,-0.003656,-0.006298,-55.0,-95.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-24,16875.0,16859.0,16857.0,16884.0,16844.0,16835.666667,16829.0,32.145503,24.494897,0.000594,0.002378,10.0,40.0
2025-04-25,16799.0,16864.0,16875.0,16884.0,16849.0,16857.333333,16838.0,7.637626,27.928480,0.000297,0.003869,5.0,65.0
2025-04-28,16824.0,16824.0,16799.0,16879.0,16799.0,16849.000000,16839.0,21.794495,27.156951,-0.002372,-0.001484,-40.0,-25.0
2025-04-29,16857.0,16849.0,16824.0,16863.0,16819.0,16845.666667,16849.0,20.207259,15.411035,0.001486,-0.000593,25.0,-10.0


In [22]:
# Daftar model
models = {
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.01),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42)
}

# Evaluasi semua model
results = []
predictions = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions[name] = y_pred
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)
    results.append({
        'Model': name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2
    })

# Buat DataFrame hasil evaluasi
results_df = pd.DataFrame(results)
results_df.round(4)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.435e+06, tolerance: 1.185e+05

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.437e+06, tolerance: 1.185e+05

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



,Model,MAE,MSE,RMSE,MAPE,R2
0,ElasticNet,39.7351,2.874916e+03,53.6182,0.2528,0.9895
1,LinearRegression,40.1994,2.950131e+03,54.3151,0.2556,0.9892
2,Ridge,39.4326,2.857432e+03,53.4550,0.2508,0.9896
3,Lasso,39.7355,2.874964e+03,53.6187,0.2528,0.9895
4,BayesianRidge,39.3793,2.857812e+03,53.4585,0.2505,0.9896
5,HuberRegressor,38.3154,2.784377e+03,52.7672,0.2437,0.9898
6,RandomForest,104.2198,1.934859e+04,139.0992,0.6525,0.9294
7,GradientBoosting,72.9179,1.293850e+04,113.7475,0.4566,0.9528
8,AdaBoost,137.1125,3.079296e+04,175.4792,0.8659,0.8877
9,ExtraTrees,88.8501,1.423573e+04,119.3136,0.5558,0.9481


In [23]:
pd.DataFrame({'Actual': y_test, 'Predicted': predictions['LinearRegression']})

,Actual,Predicted
2023-03-28,15084.0,15142.937998
2023-03-29,15059.0,15058.388725
2023-03-30,15044.0,15055.683482
2023-03-31,14989.0,15046.508517
2023-04-03,14964.0,15007.184268
...,...,...
2025-04-24,16864.0,16862.335388
2025-04-25,16824.0,16823.405357
2025-04-28,16849.0,16820.573381
2025-04-29,16754.0,16841.089286


In [24]:
# Inisialisasi figure
fig = go.Figure()

# Tambahkan garis aktual
fig.add_trace(go.Scatter(
    x=y_test.index,
    y=y_test.values,
    mode='lines',
    name='Actual',
    line=dict(width=2),
    visible=True  # selalu terlihat
))

# Tambahkan garis prediksi dari setiap model, awalnya diset tidak terlihat
for i, (name, y_pred) in enumerate(predictions.items()):
    fig.add_trace(go.Scatter(
        x=y_test.index,
        y=y_pred,
        mode='lines',
        name=f'Predicted - {name}',
        line=dict(width=2),
        visible=False  # default tidak terlihat
    ))

# Set prediksi pertama agar terlihat secara default
fig.data[1].visible = True  # trace ke-1 = prediksi pertama

# Buat tombol dropdown
dropdown_buttons = []
n_models = len(predictions)
for i, name in enumerate(predictions.keys()):
    # Visibility: True untuk trace aktual + hanya prediksi i yang True
    visibility = [True] + [j == i for j in range(n_models)]
    dropdown_buttons.append(dict(
        label=name,
        method='update',
        args=[
            {'visible': visibility},
            {'title': f'{name} Prediction of USD/IDR Closing Price'}
        ]
    ))

# Update layout dengan dropdown
fig.update_layout(
    title='Model Prediction of USD/IDR Closing Price',
    xaxis_title='Date',
    yaxis_title='Closing Price (IDR)',
    legend=dict(x=0.01, y=0.99),
    template='plotly_white',
    width=1000,
    height=500,
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction="down",
        showactive=True,
        x=1.15,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )]
)

# Tampilkan grafik
fig.show()

In [31]:
# Pilih model terbaik secara manual
best_model_name = 'LinearRegression'  # ← Ganti sesuai model yang Anda pilih
best_model = models[best_model_name]

In [32]:
X_latest = latest_row[[
    'Open_today', 'Close_t-1', 'Open_t-1', 'High_t-1', 'Low_t-1',
    'MA_3', 'MA_5', 'STD_3', 'STD_5', 'ROC_1', 'ROC_3', 'Momentum_1', 'Momentum_3'
]]
X_latest

,Open_today,Close_t-1,Open_t-1,High_t-1,Low_t-1,MA_3,MA_5,STD_3,STD_5,ROC_1,ROC_3,Momentum_1,Momentum_3
2025-05-01,16674.0,16594.0,16782.0,16782.0,16589.0,16732.333333,16777.0,128.873323,110.657128,-0.00955,-0.013671,-160.0,-230.0


In [33]:
# 🔮 Prediksi hari terakhir (yang tadi dipisahkan)

y_actual = latest_row['Close'].values[0]
y_forecast = best_model.predict(X_latest)[0]
absolute_error = abs(y_forecast - y_actual)

print("\nReal-time Forecast (Simulasi Hari Terakhir):")
print(f"Tanggal         : {latest_row.index[0].date()}")
print(f"Actual Close    : {y_actual:.2f}")
print(f"Predicted Close : {y_forecast:.2f}")
print(f"Absolute Error   : {absolute_error:.2f}")


Real-time Forecast (Simulasi Hari Terakhir):
Tanggal         : 2025-05-01
Actual Close    : 16597.00
Predicted Close : 16590.29
Absolute Error   : 6.71
